# Elliptic PDEs

The classic example of an elliptic PDE is **Laplace's equation** (yep, the same Laplace that gave us the Laplace transform), which in two dimensions for a variable $u(x,y)$ is
\begin{equation}
\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = \nabla^2 u = 0 \;,
\end{equation}
where $\nabla$ is del, or nabla, and represents the gradient operator: $\nabla = \frac{\partial}{\partial x} + \frac{\partial}{\partial y}$.

Laplace's equation shows up in a number of physical problems, including heat transfer, fluid dynamics, and electrostatics. For example, the heat equation for conduction in two dimensions is
\begin{equation}
\frac{\partial u}{\partial t} = \alpha \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right) \;,
\end{equation}
where $u(x,y,t)$ is temperature and $\alpha$ is thermal diffusivity. Steady-state heat transfer (meaning after any initial transient period) is then described by Laplace's equation.

A related elliptic PDE is **Poisson's equation**:
\begin{equation}
\nabla^2 u = f(x,y) \;,
\end{equation}
which also appears in multiple physical problems—most notably, when solving for pressure in the Navier–Stokes equations.

To numerically solve these equations, and any elliptic PDE, we can use finite differences, where we replace the continuous $x,y$ domain with a discrete grid of points. This is similar to what we did with boundary-value problems in one dimension—but now we have two dimensions.

To approximate the second derivatives in Laplace's equation, we can use central differences in both the $x$ and $y$ directions, applied around the $u_{i,j}$ point:
\begin{align}
\frac{\partial^2 u}{\partial x^2} &\approx \frac{u_{i-1,j} - 2u_{i,j} + u_{i+1,j}}{\Delta x^2} \\
\frac{\partial^2 u}{\partial y^2} &\approx \frac{u_{i,j-1} - 2u_{i,j} + u_{i,j+1}}{\Delta y^2}
\end{align}
where $i$ is the index used in the $x$ direction, $j$ is the index in the $y$ direction, and $\Delta x$ and $\Delta y$ are the step sizes in the $x$ and $y$ directions.
In other words, $x_i = (i-1) \Delta x$ and $y_j = (j-1) \Delta y$.

The following figure shows the points necessary to approximate the partial derivatives in the PDE at a location $(x_i, y_j)$, for a general 2D region. This is known as a **five-point stencil**:
<figure>
  <center>
  <img src="../images/five-point-stencil.png" alt="five-point stencil" style="width: 350px;"/>
  <figcaption>Figure: Five-point finite difference stencil</figcaption>
  </center>
</figure>

Applying these finite differences gives us an approximation for Laplace's equation:
\begin{equation}
\frac{u_{i-1,j} - 2u_{i,j} + u_{i+1,j}}{\Delta x^2} + \frac{u_{i,j-1} - 2u_{i,j} + u_{i,j+1}}{\Delta y^2} = 0 \;.
\end{equation}
If we use a uniform grid where $\Delta x = \Delta y = h$, then we can simplify to 
\begin{equation}
u_{i+1,j} + u_{i,j+1} + u_{i-1,j} + u_{i,j-1} - 4 u_{i,j} = 0 \;.
\end{equation}

## Example: heat transfer in a square plate

As an example, let's consider the problem of steady-state heat transfer in a square solid object. If $u(x,y)$ is temperature, then this is described by Laplace's equation:
\begin{equation}
\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = \nabla^2 u = 0 \;,
\end{equation}
and we can solve this using finite differences. Using a uniform grid where $\Delta x = \Delta y = h$, Laplace's equation gives us a recursion formula that relates the values at neighboring points:
\begin{equation}
u_{i+1,j} + u_{i,j+1} + u_{i-1,j} + u_{i,j-1} - 4 u_{i,j} = 0 \;.
\end{equation}

Consider a case where the square has sides of length $L$, and the boundary conditions are that the temperature is fixed at 100 on the left, right, and bottom sides, and fixed at 0 on the top.
For now, we'll use three segments to discretize the domain in both directions.
The following figures show the example problem, and the grid of points we'll use.

<table><tr>
<td> 
    <figure><center>
        <img src="../images/heat-transfer-square.png" alt="Heat transfer in a square" style="width: 250px;"/> 
        <figcaption>Figure: Heat transfer in a square object</figcaption>
    </center></figure>
</td>
<td> 
    <figure><center>
        <img src="../images/heat-equation-grid.png" alt="4x4 grid" style="width: 250px;"/> 
        <figcaption>Figure: Simple 4x4 grid of points</figcaption>
    </center></figure>
</td>
</tr></table>

Using the above recursion formula, we can write an equation for each of the four unknown points (in the interior, not the boundary points):
\begin{align}
\text{for } u_{1,1}: \quad u_{2,1} + u_{1,2} + u_{0,1} + u_{1,0} - 4u_{1,1} &= 0 \\
\text{for } u_{2,1}: \quad u_{3,1} + u_{2,2} + u_{1,1} + u_{2,0} - 4u_{2,1} &= 0 \\
\text{for } u_{1,2}: \quad u_{2,2} + u_{1,3} + u_{0,2} + u_{1,1} - 4u_{1,2} &= 0 \\
\text{for } u_{2,2}: \quad u_{3,2} + u_{2,3} + u_{1,2} + u_{2,1} - 4u_{2,2} &= 0
\end{align}
where $u_{1,1}$, $u_{2,1}$, $u_{1,2}$, and $u_{2,2}$ are the unknowns, and $u_{0,j}$, $u_{3,j}$, $u_{i,0}$, and $u_{i,3}$ are known boundary values.

Let's rearrange this system of equations slightly, to move all the constant terms to the right-hand side. These constants come from our boundary conditions, where $u_{1,0} = u_{2,0} = 100$, $u_{0,1} = u_{0,2} = 100$, $u_{3,1} = u_{3,2} = 100$, and $u_{1,3} = u_{2,3} = 0$. (The points in the corners do not play a role here.)
\begin{align}
-4u_{1,1} + u_{2,1} + u_{1,2} &= -200 \\
u_{1,1} - 4u_{2,1} + u_{2,2} &= -200 \\
u_{1,1} - 4u_{1,2} + u_{2,2} &= -100 \\
u_{2,1} + u_{1,2} - 4u_{2,2} &= -100
\end{align}
This is a system of linear equations, that we can represent as a matrix-vector product:
\begin{align}
\begin{bmatrix} -4 & 1 & 1 & 0 \\
1 & -4 & 0 & 1 \\
1 & 0 & -4 & 1 \\
0 & 1 & 1 & -4 \end{bmatrix}
\begin{bmatrix} u_{1,1} \\ u_{2,1} \\ u_{1,2} \\ u_{2,2} \end{bmatrix} &= 
\begin{bmatrix} -200 \\ -200 \\ -100 \\ -100 \end{bmatrix} \\
\text{or} \quad A \mathbf{u} &= \mathbf{b}
\end{align}
where $A$ is a $4\times 4$ coefficient matrix, $\mathbf{u}$ is a four-element vector of unknown variables, and $\mathbf{b}$ is a four-element right-hand side vector.
For $\mathbf{u}$, we had to take variables that physically represent points in a two-dimensional space and combine them in some order to form a one-dimensional column vector. Here, we used a **row-major** mapping, where we started with the point in the first row and first column, then added the remaining points in that row, before moving to the next row and repeating. We'll discuss this a bit more later.

If we set this up in Matlab, we can solve with `u = A \ b`:

In [3]:
clear all; clc

A = [-4  1  1  0; 
      1 -4  0  1;
      1  0 -4  1;
      0  1  1 -4];
b = [-200; -200; -100; -100];

u = A \ b;

disp(u)

   87.5000
   87.5000
   62.5000
   62.5000



This gives us the values for temperature at each of the four points. As a quick check, they all fall between 0 and 100, with the colder points on the second row, near the top boundary where the temperature was fixed at 0.